<a href="https://colab.research.google.com/github/PDas2206/BERT_emotion_analysis/blob/master/friends_finetune_31.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [1]:
!pip install transformers
import pandas as pd
import numpy as np
import tensorflow as tf
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, BCELoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, f1_score, accuracy_score
import pickle
from transformers import *
from tqdm import tqdm, trange
from ast import literal_eval

     |████████████████████████████████| 1.3MB 3.4MB/s 
     |████████████████████████████████| 2.9MB 14.3MB/s 
     |████████████████████████████████| 890kB 40.3MB/s 
     |████████████████████████████████| 1.1MB 43.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=7cc69b3759f82e31879e412a81569ae211f41e655dc987d2a75a1ed7c5273099
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
 from google.colab import drive
 drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla P4'

## Load and Preprocess Training Data

Dataset will be tokenized then split into training and validation sets. The validation set will be used to monitor training. For testing a separate test set will be loaded for analysis.

In [5]:
df = pd.read_csv('/content/drive/My Drive/friends_train_nofilter.csv') #jigsaw-toxic-comment-classification-challenge
df.head()

,annotation,speaker,utterance,neutral,anger,disgust,fear,joy,sadness,surprise
0,4100000,Chandler,also I was the point person on my companys tr...,1,0,0,0,0,0,0
1,5000000,The Interviewer,You mustve had your hands full.,1,0,0,0,0,0,0
2,5000000,Chandler,That I did. That I did.,1,0,0,0,0,0,0
3,5000000,The Interviewer,So lets talk a little bit about your duties.,1,0,0,0,0,0,0
4,2000030,Chandler,My duties? All right.,0,0,0,0,0,0,1


In [6]:
print('Unique comments: ', df.utterance.nunique() == df.shape[0])
print('Null values: ', df.isnull().values.any())
# df[df.isna().any(axis=1)]

Unique comments:  False
Null values:  False


In [7]:
print('average sentence length: ', df.utterance.str.split().str.len().mean())
print('stdev sentence length: ', df.utterance.str.split().str.len().std())

average sentence length:  7.792727961367294
stdev sentence length:  6.231603847035365


In [8]:
cols = df.columns
label_cols = list(cols[3:])
num_labels = len(label_cols)
print('Label columns: ', label_cols)

Label columns:  ['neutral', 'anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']


In [9]:
print('Count of 1 per label: \n', df[label_cols].sum(), '\n') # Label counts, may need to downsample or upsample
print('Count of 0 per label: \n', df[label_cols].eq(0).sum())

Count of 1 per label: 
 neutral     4752
anger        513
disgust      240
fear         185
joy         1283
sadness      351
surprise    1220
dtype: int64 

Count of 0 per label: 
 neutral      5809
anger       10048
disgust     10321
fear        10376
joy          9278
sadness     10210
surprise     9341
dtype: int64


In [10]:
df = df.sample(frac=1).reset_index(drop=True) #shuffle rows

In [11]:
df['one_hot_labels'] = list(df[label_cols].values)
df.head()

,annotation,speaker,utterance,neutral,anger,disgust,fear,joy,sadness,surprise,one_hot_labels
0,5000000,Lydia,The Celtics.,1,0,0,0,0,0,0,"[1, 0, 0, 0, 0, 0, 0]"
1,5000000,Monica,I,1,0,0,0,0,0,0,"[1, 0, 0, 0, 0, 0, 0]"
2,5000000,Chandler,Check this out.,1,0,0,0,0,0,0,"[1, 0, 0, 0, 0, 0, 0]"
3,1010003,Monica,Can we turn the TV off? Okay? Do we really wan...,0,0,1,0,0,0,0,"[0, 0, 1, 0, 0, 0, 0]"
4,4000001,Mark,Its for me.,1,0,0,0,0,0,0,"[1, 0, 0, 0, 0, 0, 0]"


In [12]:
labels = list(df.one_hot_labels.values)
comments = list(df.utterance.values)

Load the pretrained tokenizer that corresponds to your choice in model. e.g.,

```
BERT:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True) 

XLNet:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=False) 

RoBERTa:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=False)
```


In order to avoid memory issues with Google Colab, I enforce a max_length of 100 tokens. Note that some sentences may not adequately represent each label because of this.

In [13]:
max_length = 100
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True) # tokenizer
encodings = tokenizer.batch_encode_plus(comments,max_length=max_length,pad_to_max_length=True) # tokenizer's encoding method
print('tokenizer outputs: ', encodings.keys())

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


tokenizer outputs:  dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [14]:
input_ids = encodings['input_ids'] # tokenized and encoded sentences
token_type_ids = encodings['token_type_ids'] # token type ids
attention_masks = encodings['attention_mask'] # attention masks

In [15]:
# Identifying indices of 'one_hot_labels' entries that only occur once - this will allow us to stratify split our training data later
label_counts = df.one_hot_labels.astype(str).value_counts()
one_freq = label_counts[label_counts==1].keys()
one_freq_idxs = sorted(list(df[df.one_hot_labels.astype(str).isin(one_freq)].index), reverse=True)
print('df label indices with only one instance: ', one_freq_idxs)

df label indices with only one instance:  []


In [16]:
# Gathering single instance inputs to force into the training set after stratified split
one_freq_input_ids = [input_ids.pop(i) for i in one_freq_idxs]
one_freq_token_types = [token_type_ids.pop(i) for i in one_freq_idxs]
one_freq_attention_masks = [attention_masks.pop(i) for i in one_freq_idxs]
one_freq_labels = [labels.pop(i) for i in one_freq_idxs]

Tokenizing the dev set for validation purposes

In [17]:
df_v = pd.read_csv('/content/drive/My Drive/friends_dev_nofilter.csv') #jigsaw-toxic-comment-classification-challenge
df_v.head()
print('Unique comments: ', df_v.utterance.nunique() == df_v.shape[0])
print('Null values: ', df_v.isnull().values.any())
# df[df.isna().any(axis=1)]
print('average sentence length: ', df_v.utterance.str.split().str.len().mean())
print('stdev sentence length: ', df_v.utterance.str.split().str.len().std())
cols = df_v.columns
label_cols = list(cols[3:])
num_labels = len(label_cols)
print('Label columns: ', label_cols)
print('Count of 1 per label: \n', df_v[label_cols].sum(), '\n') # Label counts, may need to downsample or upsample
print('Count of 0 per label: \n', df_v[label_cols].eq(0).sum())
df_v = df_v.sample(frac=1).reset_index(drop=True) #shuffle rows
df_v['one_hot_labels'] = list(df_v[label_cols].values)
df_v.head()

validation_ids = encodings['input_ids'] # tokenized and encoded sentences

Unique comments:  False
Null values:  False
average sentence length:  7.703735144312394
stdev sentence length:  5.996005719851644
Label columns:  ['neutral', 'anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']
Count of 1 per label: 
 neutral     491
anger        85
disgust      23
fear         29
joy         123
sadness      62
surprise    151
dtype: int64 

Count of 0 per label: 
 neutral      687
anger       1093
disgust     1155
fear        1149
joy         1055
sadness     1116
surprise    1027
dtype: int64


Be sure to handle all classes during validation using "stratify" during train/validation split:

In [18]:
# Use train_test_split to split our data into train and validation sets

#train_inputs, validation_inputs, train_labels, validation_labels, train_token_types, validation_token_types, train_masks, validation_masks = train_test_split(input_ids, labels, token_type_ids,attention_masks,
#                                                           random_state=2020, test_size=0.10, stratify = labels)
train_inputs=input_ids
train_labels=labels
train_token_types=token_type_ids
train_masks=attention_masks


validation_inputs=validation_ids
validation_labels=labels
validation_token_types=token_type_ids
validation_masks=attention_masks

# Add one frequency data to train data
train_inputs.extend(one_freq_input_ids)
train_labels.extend(one_freq_labels)
train_masks.extend(one_freq_attention_masks)
train_token_types.extend(one_freq_token_types)

# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
train_token_types = torch.tensor(train_token_types)

validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)
validation_token_types = torch.tensor(validation_token_types)

In [19]:
# Select a batch size for training. For fine-tuning with XLNet, the authors recommend a batch size of 32, 48, or 128. We will use 32 here to avoid memory issues.
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels, train_token_types)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels, validation_token_types)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [20]:
torch.save(validation_dataloader,'validation_data_loader')
torch.save(train_dataloader,'train_data_loader')

## Load Model & Set Params

Load the appropriate model below, each model already contains a single dense layer for classification on top.



```
BERT:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

XLNet:
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=num_labels)

RoBERTa:
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=num_labels)
```



In [21]:
# Load model, the pretrained model will include a single linear classification layer on top for classification. 
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

Setting custom optimization parameters for the AdamW optimizer https://huggingface.co/transformers/main_classes/optimizer_schedules.html

In [22]:
# setting custom optimization parameters. You may implement a scheduler here as well.
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [23]:
optimizer = AdamW(optimizer_grouped_parameters,lr=2e-5,correct_bias=True)
# optimizer = AdamW(model.parameters(),lr=2e-5)  # Default optimization

## Train Model

In [25]:
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 3

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):

  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()

  # Tracking variables
  tr_loss = 0 #running loss
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels, b_token_types = batch
    #print("labels: ",b_labels)
    print("num of labels: ",num_labels)
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()

    # # Forward pass for multiclass classification
    #outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    #target = Variable(torch.LongTensor([7]))
    #outputs.shape
    #loss = outputs[0]
    #loss=outputs.loss
    #logits = outputs[1]
    #loss_func=nn.CrossEntropyLoss()
    #print(b_labels.view(-1,num_labels))
    #loss=loss_func(torch.sigmoid(logits.view(-1,num_labels)),b_labels.view(-1))
    #loss = loss_func(logits.view(-1,num_labels),b_labels.view(-1,num_labels))

    # Forward pass for multilabel classification
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]
    loss_func = BCEWithLogitsLoss() 
    loss = loss_func(logits.view(-1,num_labels),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation
    # loss_func = BCELoss() 
    # loss = loss_func(torch.sigmoid(logits.view(-1,num_labels)),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation
    train_loss_set.append(loss.item())    

    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    # scheduler.step()
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))

###############################################################################

  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Variables to gather full output
  logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

  # Predict
  for i, batch in enumerate(validation_dataloader):
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels, b_token_types = batch
    with torch.no_grad():
      # Forward pass
      outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      b_logit_pred = outs[0]
      pred_label = torch.sigmoid(b_logit_pred)

      b_logit_pred = b_logit_pred.detach().cpu().numpy()
      pred_label = pred_label.to('cpu').numpy()
      b_labels = b_labels.to('cpu').numpy()

    tokenized_texts.append(b_input_ids)
    logit_preds.append(b_logit_pred)
    true_labels.append(b_labels)
    pred_labels.append(pred_label)

  # Flatten outputs
  pred_labels = [item for sublist in pred_labels for item in sublist]
  true_labels = [item for sublist in true_labels for item in sublist]

  # Calculate Accuracy
  threshold = 0.50
  pred_bools = [pl>threshold for pl in pred_labels]
  true_bools = [tl==1 for tl in true_labels]
  val_f1_accuracy = f1_score(true_bools,pred_bools,average='micro')*100
  val_flat_accuracy = accuracy_score(true_bools, pred_bools)*100

  print('F1 Validation Accuracy: ', val_f1_accuracy)
  print('Flat Validation Accuracy: ', val_flat_accuracy)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of lab

Epoch:  33%|███▎      | 1/3 [04:27<08:54, 267.25s/it]

F1 Validation Accuracy:  67.03077128013854
Flat Validation Accuracy:  60.04166272133321
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of label

Epoch:  67%|██████▋   | 2/3 [08:56<04:27, 267.93s/it]

F1 Validation Accuracy:  74.41705529646903
Flat Validation Accuracy:  67.50307736009847
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of labels:  7
num of label

Epoch: 100%|██████████| 3/3 [13:26<00:00, 268.72s/it]

F1 Validation Accuracy:  82.50338128611828
Flat Validation Accuracy:  76.64993845279803


In [26]:
torch.save(model.state_dict(), 'bert_model_toxic')

## Load and Preprocess Test Data

In [27]:
test_df = pd.read_csv('/content/drive/My Drive/friends_test_nofilter.csv')
#test_labels_df = pd.read_csv('test_labels.csv')
#test_df = test_df.merge(test_labels_df, on='id', how='left')
test_label_cols = list(test_df.columns[3:])
print('Null values: ', test_df.isnull().values.any()) #should not be any null sentences or labels
print('Same columns between train and test: ', label_cols == test_label_cols) #columns should be the same
test_df.head()

Null values:  False
Same columns between train and test:  True


,annotation,speaker,utterance,neutral,anger,disgust,fear,joy,sadness,surprise
0,2000030,Mark,Why do all youre coffee mugs have numbers on ...,0,0,0,0,0,0,1
1,2100011,Rachel,Oh. Thats so Monica can keep track. That way ...,0,0,0,0,0,0,0
2,3000020,Rachel,Y'know what?,1,0,0,0,0,0,0
3,5000000,Ross,It didnt.,1,0,0,0,0,0,0
4,1300010,Frank,"Okay, so what you used to have with Rachel, is...",0,0,0,0,1,0,0


In [28]:
test_df = test_df[~test_df[test_label_cols].eq(-1).any(axis=1)] #remove irrelevant rows/comments with -1 values
test_df['one_hot_labels'] = list(test_df[test_label_cols].values)
test_df.head()

,annotation,speaker,utterance,neutral,anger,disgust,fear,joy,sadness,surprise,one_hot_labels
0,2000030,Mark,Why do all youre coffee mugs have numbers on ...,0,0,0,0,0,0,1,"[0, 0, 0, 0, 0, 0, 1]"
1,2100011,Rachel,Oh. Thats so Monica can keep track. That way ...,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0]"
2,3000020,Rachel,Y'know what?,1,0,0,0,0,0,0,"[1, 0, 0, 0, 0, 0, 0]"
3,5000000,Ross,It didnt.,1,0,0,0,0,0,0,"[1, 0, 0, 0, 0, 0, 0]"
4,1300010,Frank,"Okay, so what you used to have with Rachel, is...",0,0,0,0,1,0,0,"[0, 0, 0, 0, 1, 0, 0]"


In [29]:
# Gathering input data
test_labels = list(test_df.one_hot_labels.values)
test_comments = list(test_df.utterance.values)

In [30]:
# Encoding input data
test_encodings = tokenizer.batch_encode_plus(test_comments,max_length=max_length,pad_to_max_length=True)
test_input_ids = test_encodings['input_ids']
test_token_type_ids = test_encodings['token_type_ids']
test_attention_masks = test_encodings['attention_mask']

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [31]:
# Make tensors out of data
test_inputs = torch.tensor(test_input_ids)
test_labels = torch.tensor(test_labels)
test_masks = torch.tensor(test_attention_masks)
test_token_types = torch.tensor(test_token_type_ids)
# Create test dataloader
test_data = TensorDataset(test_inputs, test_masks, test_labels, test_token_types)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)
# Save test dataloader
torch.save(test_dataloader,'test_data_loader')

## Prediction and Metics

In [32]:
# Test

# Put model in evaluation mode to evaluate loss on the validation set
model.eval()

#track variables
logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

# Predict
for i, batch in enumerate(test_dataloader):
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels, b_token_types = batch
  with torch.no_grad():
    # Forward pass
    outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    b_logit_pred = outs[0]
    pred_label = torch.sigmoid(b_logit_pred)

    b_logit_pred = b_logit_pred.detach().cpu().numpy()
    pred_label = pred_label.to('cpu').numpy()
    b_labels = b_labels.to('cpu').numpy()

  tokenized_texts.append(b_input_ids)
  logit_preds.append(b_logit_pred)
  true_labels.append(b_labels)
  pred_labels.append(pred_label)

# Flatten outputs
tokenized_texts = [item for sublist in tokenized_texts for item in sublist]
pred_labels = [item for sublist in pred_labels for item in sublist]
true_labels = [item for sublist in true_labels for item in sublist]
# Converting flattened binary values to boolean values
true_bools = [tl==1 for tl in true_labels]

We need to threshold our sigmoid function outputs which range from [0, 1]. Below I use 0.50 as a threshold.

In [33]:
pred_bools = [pl>0.50 for pl in pred_labels] #boolean output after thresholding

# Print and save classification report
print('Test F1 Accuracy: ', f1_score(true_bools, pred_bools,average='micro'))
print('Test Flat Accuracy: ', accuracy_score(true_bools, pred_bools),'\n')
clf_report = classification_report(true_bools,pred_bools,target_names=test_label_cols)
pickle.dump(clf_report, open('/content/drive/My Drive/classification_report_nofilter_joy.txt','wb')) #save report
print(clf_report)

Test F1 Accuracy:  0.654871546291808
Test Flat Accuracy:  0.5886396526772794 



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

     neutral       0.78      0.75      0.76      1287
       anger       0.52      0.31      0.39       161
     disgust       0.00      0.00      0.00        68
        fear       0.00      0.00      0.00        32
         joy       0.76      0.44      0.56       304
     sadness       0.91      0.12      0.21        85
    surprise       0.51      0.67      0.58       286

   micro avg       0.71      0.61      0.65      2223
   macro avg       0.50      0.33      0.36      2223
weighted avg       0.69      0.61      0.63      2223
 samples avg       0.49      0.49      0.49      2223



## Output Dataframe

In [34]:
idx2label = dict(zip(range(7),label_cols))
print(idx2label)

{0: 'neutral', 1: 'anger', 2: 'disgust', 3: 'fear', 4: 'joy', 5: 'sadness', 6: 'surprise'}


In [35]:
# Getting indices of where boolean one hot vector true_bools is True so we can use idx2label to gather label names
true_label_idxs, pred_label_idxs=[],[]
for vals in true_bools:
  true_label_idxs.append(np.where(vals)[0].flatten().tolist())
for vals in pred_bools:
  pred_label_idxs.append(np.where(vals)[0].flatten().tolist())

In [36]:
# Gathering vectors of label names using idx2label
true_label_texts, pred_label_texts = [], []
for vals in true_label_idxs:
  if vals:
    true_label_texts.append([idx2label[val] for val in vals])
  else:
    true_label_texts.append(vals)

for vals in pred_label_idxs:
  if vals:
    pred_label_texts.append([idx2label[val] for val in vals])
  else:
    pred_label_texts.append(vals)

In [37]:
# Decoding input ids to comment text
utterances = [tokenizer.decode(text,skip_special_tokens=True,clean_up_tokenization_spaces=False) for text in tokenized_texts]

In [38]:
# Converting lists to df
comparisons_df = pd.DataFrame({'utterance': utterances, 'true_labels': true_label_texts, 'pred_labels':pred_label_texts})
comparisons_df.to_csv('/content/drive/My Drive/comparisons_nofilter_joy.csv')
comparisons_df.head()

,utterance,true_labels,pred_labels
0,why do all youre coffee mugs have numbers on t...,[surprise],[]
1,oh . thats so monica can keep track . that way...,[],[neutral]
2,y ' know what ?,[neutral],[neutral]
3,it didnt .,[neutral],[neutral]
4,"okay , so what you used to have with rachel , ...",[joy],[neutral]


## Bonus - Optimizing threshold value for micro F1 score

Doing this may result in a trade offs between precision, flat accuracy and micro F1 accuracy. You may tune the threshold however you want.

In [39]:
# Calculate Accuracy - maximize F1 accuracy by tuning threshold values. First with 'macro_thresholds' on the order of e^-1 then with 'micro_thresholds' on the order of e^-2

macro_thresholds = np.array(range(1,10))/10

f1_results, flat_acc_results = [], []
for th in macro_thresholds:
  pred_bools = [pl>th for pl in pred_labels]
  test_f1_accuracy = f1_score(true_bools,pred_bools,average='micro')
  test_flat_accuracy = accuracy_score(true_bools, pred_bools)
  f1_results.append(test_f1_accuracy)
  flat_acc_results.append(test_flat_accuracy)

best_macro_th = macro_thresholds[np.argmax(f1_results)] #best macro threshold value

micro_thresholds = (np.array(range(10))/100)+best_macro_th #calculating micro threshold values

f1_results, flat_acc_results = [], []
for th in micro_thresholds:
  pred_bools = [pl>th for pl in pred_labels]
  test_f1_accuracy = f1_score(true_bools,pred_bools,average='micro')
  test_flat_accuracy = accuracy_score(true_bools, pred_bools)
  f1_results.append(test_f1_accuracy)
  flat_acc_results.append(test_flat_accuracy)

best_f1_idx = np.argmax(f1_results) #best threshold value

# Printing and saving classification report
print('Best Threshold: ', micro_thresholds[best_f1_idx])
print('Test F1 Accuracy: ', f1_results[best_f1_idx])
print('Test Flat Accuracy: ', flat_acc_results[best_f1_idx], '\n')

best_pred_bools = [pl>micro_thresholds[best_f1_idx] for pl in pred_labels]
clf_report_optimized = classification_report(true_bools,best_pred_bools, target_names=label_cols)
pickle.dump(clf_report_optimized, open('/content/drive/My Drive/classification_report_optimized.txt','wb'))
print(clf_report_optimized)

Best Threshold:  0.3
Test F1 Accuracy:  0.6681175190424374
Test Flat Accuracy:  0.599493487698987 



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

     neutral       0.74      0.81      0.78      1287
       anger       0.40      0.49      0.44       161
     disgust       0.00      0.00      0.00        68
        fear       0.00      0.00      0.00        32
         joy       0.66      0.60      0.63       304
     sadness       0.65      0.24      0.34        85
    surprise       0.45      0.74      0.56       286

   micro avg       0.65      0.69      0.67      2223
   macro avg       0.41      0.41      0.39      2223
weighted avg       0.63      0.69      0.65      2223
 samples avg       0.55      0.56      0.55      2223

